<a href="https://colab.research.google.com/github/SanchitaMahajan/Programming-Assignment/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install requirements
!pip install kora -q

#load packages
from bs4 import BeautifulSoup
from kora.selenium import wd

     |████████████████████████████████| 57 kB 2.9 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 


In [ ]:
# setup the URL
URL = 'https://au.finance.yahoo.com/quote/NAB.AX/profile?p=NAB.AX'

# Get the page
wd.get(URL)

In [ ]:
soup = BeautifulSoup(wd.page_source, "html.parser")

tables = soup.find_all('table')
len(tables)

1

In [ ]:
tbody = tables[0].find('tbody')

In [ ]:
rows = tbody.find_all('tr')

In [ ]:
columns = rows[0].find_all('td')

In [ ]:
columns[0].text

'Mr. Ross Maxwell McEwan CBE, B.B.S.'

In [ ]:
tables = soup.find_all('table')
for table in tables:
  body = table.find('tbody')
  rows = body.find_all("tr")
  for row in rows:
    data = row.find_all('td')
    for txt in data:
      print(txt.text)
    print()

Mr. Ross Maxwell McEwan CBE, B.B.S.
Group CEO, MD & Director
2.54M
N/A
1957

Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons), FCA
Group Chief Financial Officer
1.13M
N/A
N/A

Mr. Patrick F. Wright
Group Exec. of Technology & Enterprise Operations
1.72M
N/A
1967

Ms. Sharon J. Cook
Group Exec. of Legal & Commercial Services
912.98k
N/A
N/A

Ms. Susan  Ferrier
Group Exec. of People & Culture
929.27k
N/A
N/A

Ms. Angela  Mentis B.Bus
Group Chief Digital, Data & Analytics Officer
1.62M
N/A
N/A

Mr. Shaun  Dooley
Chief Risk Officer
1.05M
N/A
N/A

Mr. Andrew  Irvine
Chief Exec. of Bus. Private Banking Division
333.27k
N/A
N/A

Mr. Lesile D. Matheson
Group Chief Operating Officer
N/A
N/A
1960

Ms. Sally  Mihell
Head of Investor Relations
N/A
N/A
N/A



In [ ]:
tables = soup.find_all('table')
#for table in tables:
body = tables[0].find('tbody')
rows = body.find_all("tr")
for row in rows:
  data = row.find_all('td')
  for txt in data:
    print(txt.text)
  print()

Mr. Ross Maxwell McEwan CBE, B.B.S.
Group CEO, MD & Director
2.54M
N/A
1957

Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons), FCA
Group Chief Financial Officer
1.13M
N/A
N/A

Mr. Patrick F. Wright
Group Exec. of Technology & Enterprise Operations
1.72M
N/A
1967

Ms. Sharon J. Cook
Group Exec. of Legal & Commercial Services
912.98k
N/A
N/A

Ms. Susan  Ferrier
Group Exec. of People & Culture
929.27k
N/A
N/A

Ms. Angela  Mentis B.Bus
Group Chief Digital, Data & Analytics Officer
1.62M
N/A
N/A

Mr. Shaun  Dooley
Chief Risk Officer
1.05M
N/A
N/A

Mr. Andrew  Irvine
Chief Exec. of Bus. Private Banking Division
333.27k
N/A
N/A

Mr. Lesile D. Matheson
Group Chief Operating Officer
N/A
N/A
1960

Ms. Sally  Mihell
Head of Investor Relations
N/A
N/A
N/A



In [ ]:
# install requirements
!pip install kora -q

#load packages
from kora.selenium import wd
import pandas as pd

# setup the URL
URL = 'https://au.finance.yahoo.com/quote/NAB.AX/profile?p=NAB.AX'

# Get the page
wd.get(URL)

tables_as_df = pd.read_html(wd.page_source)
len(tables_as_df)

1

In [ ]:
tables_as_df[0]

,Name,Title,Pay,Exercised,Year born
0,"Mr. Ross Maxwell McEwan CBE, B.B.S.","Group CEO, MD & Director",2.54M,NaN,1957.0
1,"Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons)...",Group Chief Financial Officer,1.13M,NaN,NaN
2,Mr. Patrick F. Wright,Group Exec. of Technology & Enterprise Operations,1.72M,NaN,1967.0
3,Ms. Sharon J. Cook,Group Exec. of Legal & Commercial Services,912.98k,NaN,NaN
4,Ms. Susan Ferrier,Group Exec. of People & Culture,929.27k,NaN,NaN
5,Ms. Angela Mentis B.Bus,"Group Chief Digital, Data & Analytics Officer",1.62M,NaN,NaN
6,Mr. Shaun Dooley,Chief Risk Officer,1.05M,NaN,NaN
7,Mr. Andrew Irvine,Chief Exec. of Bus. Private Banking Division,333.27k,NaN,NaN
8,Mr. Lesile D. Matheson,Group Chief Operating Officer,NaN,NaN,1960.0
9,Ms. Sally Mihell,Head of Investor Relations,NaN,NaN,NaN


[Resident tax rates 2021–22
Taxable income](https://www.ato.gov.au/rates/individual-income-tax-rates/)

Income            | Tax on this income 
------------------|----------------------
0 – \$18,200       | Nil
\$18,201 – \$45,000 | 19 cents for each \$1 over \$18,200
\$45,001 – \$120,000 | \$5,092 plus 32.5 cents for each \$1 over \$45,000
\$120,001 – \$180,000 |  \$29,467 plus 37 cents for each \$1 over \$120,000
\$180,001 and over | \$51,667 plus 45 cents for each \$1 over \$180,000

In [ ]:
def get_Name():
  Name = input("Please input your name?")
  print("Your name is ", Name)
  return Name

In [ ]:
def get_income():
  '''
  This function will prompt the user for the income
  and return the value
  '''

  income = int(input("What is your income for the year? "))
  print("Your income is ", income)
  return income

In [ ]:
def calculate_tax(income):
  if income <= 18200:
    tax_payable = 0
  elif 18200 < income <= 45000:
    tax_payable = 0.19 * (income - 18200)
  elif 45000 < income < 120000:
    tax_payable = 5092 + 0.32 * (income - 45000)
  elif 120000 < income < 180000:
    tax_payable = 29467 + 0.37 * (income - 120000) 
  elif 180000 < income:
    tax_payable = 51667 + 0.45 * (income - 180000)

  return tax_payable

In [ ]:
# 'Main line'
Name = get_Name()
# Step 1
income = get_income()

# Step 2
tax = calculate_tax(income)

print(" Your tax payable is ", tax)

Please input your name?Rosewood Mary
Your name is  Rosewood Mary
What is your income for the year? 2054000
Your income is  2054000
 Your tax payable is  894967.0


In [1]:
stock_ticker = input("Please input the companies stock ticker: ")


template = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
url = template.format(stock_ticker,stock_ticker)


Please input the companies stock ticker: NAB.AX


In [2]:
# install requirements
!pip install kora -q

#load packages
from bs4 import BeautifulSoup
from kora.selenium import wd

     |████████████████████████████████| 57 kB 2.7 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 


In [5]:
def get_url():
  template = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
  stock_ticker = input("Please input the companies stock ticker: ")
  url = template.format(stock_ticker,stock_ticker)
  return url

In [7]:
url = get_url()
wd.get(url)


Please input the companies stock ticker: NAB.AX


In [8]:
soup = BeautifulSoup(wd.page_source, "html.parser")

tables = soup.find_all('table')
len(tables)

1

In [9]:
tbody = tables[0].find('tbody')

In [10]:
rows = tbody.find_all('tr')

In [11]:
columns = rows[0].find_all('td')

In [12]:
tables = soup.find_all('table')
for table in tables:
  body = table.find('tbody')
  rows = body.find_all("tr")
  for row in rows:
    data = row.find_all('td')
    for txt in data:
      print(txt.text)
    print()

Mr. Ross Maxwell McEwan CBE, B.B.S.
Group CEO, MD & Director
2.54M
N/A
1957

Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons), FCA
Group Chief Financial Officer
1.13M
N/A
N/A

Mr. Patrick F. Wright
Group Exec. of Technology & Enterprise Operations
1.72M
N/A
1967

Ms. Sharon J. Cook
Group Exec. of Legal & Commercial Services
912.98k
N/A
N/A

Ms. Susan  Ferrier
Group Exec. of People & Culture
929.27k
N/A
N/A

Ms. Angela  Mentis B.Bus
Group Chief Digital, Data & Analytics Officer
1.62M
N/A
N/A

Mr. Shaun  Dooley
Chief Risk Officer
1.05M
N/A
N/A

Mr. Andrew  Irvine
Chief Exec. of Bus. Private Banking Division
333.27k
N/A
N/A

Mr. Lesile D. Matheson
Group Chief Operating Officer
N/A
N/A
1960

Ms. Sally  Mihell
Head of Investor Relations
N/A
N/A
N/A



In [18]:
# install requirements
!pip install kora -q

#load packages
from bs4 import BeautifulSoup
from kora.selenium import wd

def get_url():
  template = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
  stock_ticker = input("Please input the companies stock ticker: ")
  url = template.format(stock_ticker,stock_ticker)
  return url

  url = get_url()
  wd.get(url)
  print(get_url)
   
  soup = BeautifulSoup(wd.page_source, "html.parser")
   
  tables = soup.find_all('table')
  for table in tables:
     body = table.find('tbody')
     rows = body.find_all("tr")
     for row in rows:
        data = row.find_all('td')
        for txt in data:
         print(txt.text)
         print()
